In [25]:
import pandas as pd
import numpy as np
import requests
from sklearn.metrics import f1_score, accuracy_score
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, pipeline

# Step 1

In [3]:
data = pd.read_csv("/content/test (1).tsv", sep='\t')
data= data.sample(n=10)
print(data.columns)

Index(['اين شايد اولين عزاى عمومى واقعى است كه ياد دارم. تمام وجودم عزادار است؛ پر از گريه، پر از خشم، پر از فرياد. مثل همهى شما. . #آتشنشان_تسليت', 'SAD'], dtype='object')


In [4]:
text_columnm = data.columns[0]
label_columnm = data.columns[1]

In [5]:
texts = data[text_columnm].tolist()
labels = data[label_columnm].tolist()

# Step 2

In [44]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, TFAutoModel
import torch

In [45]:
model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, torch_dtype=torch.float16)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of Phi3ForSequenceClassification were not initialized from the model checkpoint at microsoft/Phi-3-mini-4k-instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Phi3ForSequenceClassification(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (score): Linear(in_features=3072, out_f

In [52]:
input = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)

In [51]:
def predict_sentiment(model, tokenizer, texts):
  input = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)

In [ ]:
output = model(**input)

You are not running the flash-attention implementation, expect numerical differences.


# Step 3

In [ ]:
predictions = torch.argmax(outputs.logit, dim=1)

# Step 4

In [ ]:
def compute_metrics(preds, lables):
  f1 = f1_score(lables, preds)
  accuracy = accuracy_score(lables, preds)
  return f1, accuracy